<a href="https://colab.research.google.com/github/MariannaJan/CornellMovieCorpusChatbot/blob/master/CornellMovieCorpusChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip uninstall tensorflow version==1.12.0

Uninstalling tensorflow-1.12.0:
  Would remove:
    /usr/local/bin/freeze_graph
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-1.12.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
y
  Successfully uninstalled tensorflow-1.12.0
Skipping version as it is not installed.


In [3]:
!pip install https://files.pythonhosted.org/packages/48/58/b71480f9ec9d08d581d672a81b15ab5fec36a5fcda2093558a23614d8468/tensorflow-1.0.0-cp36-cp36m-manylinux1_x86_64.whl

    100% |████████████████████████████████| 44.5MB 945kB/s 
stable-baselines 2.2.1 has requirement tensorflow>=1.5.0, but you'll have tensorflow 1.0.0 which is incompatible.
magenta 0.3.19 has requirement tensorflow>=1.12.0, but you'll have tensorflow 1.0.0 which is incompatible.


In [1]:
import tensorflow as tf
print(tf.__version__)


1.0.0


In [2]:
#mounting google drive

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
#getting the Cornll Corpus Files
!wget https://www.dropbox.com/s/o0t76yphs15vjva/movie_conversations.txt?dl=0
!wget https://www.dropbox.com/s/z4n9j79vcloyt14/movie_lines.txt?dl=0  

--2019-02-02 09:40:26--  https://www.dropbox.com/s/o0t76yphs15vjva/movie_conversations.txt?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.1, 2620:100:6018:1::a27d:301
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/o0t76yphs15vjva/movie_conversations.txt [following]
--2019-02-02 09:40:26--  https://www.dropbox.com/s/raw/o0t76yphs15vjva/movie_conversations.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucff222b5e995a001e911bbe8e15.dl.dropboxusercontent.com/cd/0/inline/AahanptjJBw28eCfCHAAPCRA5--yhHeZDjisuwmtBPUvWokXVeRlA7_wfKAyDDlAjjJzx3CSJ5RhggWcyAwu1rzi-Hlmraz-NGuw-XVkeiY5tw/file# [following]
--2019-02-02 09:40:26--  https://ucff222b5e995a001e911bbe8e15.dl.dropboxusercontent.com/cd/0/inline/AahanptjJBw28eCfCHAAPCRA5--yhHeZDjisuwmtBPUvWokXVeRlA7_wfKAyDDlAjjJzx3CSJ5RhggWcyAwu1rzi-Hlmraz-N

In [0]:
#path for the chatbot files on drive

path = "/content/"

In [0]:
#reading in the conversations, returns list of lists

def read_conversations():
  filename = path + "movie_conversations.txt?dl=0"
  conversations = []
  with open(filename, "r", encoding="ISO-8859-1") as conv_file:
    for line in conv_file:
      line_str = line.split(' +++$+++ ')[-1]
      conversations.append(line_str[1:-2].replace("'","").replace(" ","").split(','))
  return conversations

In [0]:
#reading the lines, creating a dictionary  {lineId: line_text}
def read_lines():
  filename = path + "movie_lines.txt?dl=0"
  with open(filename, "r", encoding="ISO-8859-1") as lines_file:
    lines_chunks = [line.split(" +++$+++ ") for line in lines_file]
  return {line[0]: line[-1].strip() for line in lines_chunks}
    

In [0]:
#SANITIZING THE DATA


EN_WHITELIST = '0123456789abcdefghijklmnopqrstuvwxyz ' # space is included in whitelist
EN_BLACKLIST = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~\''

def remove_blacklisted(line):
  return ''.join([ch for ch in line if ch in EN_WHITELIST])

def remove_blank(line):
  return [l for l in line if l]

def prepare_line(line):
  return remove_blank(remove_blacklisted(line.lower()).split(" "))

In [0]:
#turning conversations into question-answer structure and braking them into single words - tokens

def get_tokenized_sequencial_sentences():
  conversations = read_conversations()
  line_text = read_lines()
  
  tokenized_sentences = []
  
  for conversation in conversations:
       for i in range(len(conversation) - 1):
          tokenized_sentences.append([prepare_line(line_text[conversation[i]]),prepare_line(line_text[conversation[i+1]])])
          
  return tokenized_sentences

In [0]:
#getting only sentences of the specified min / max length

MIN_LENGTH = 2
MAX_LENGTH = 25

def choose_conv_by_length():
  dataset = []
  tokenized_sentences = get_tokenized_sequencial_sentences()
  for row in tokenized_sentences:
      if ((MIN_LENGTH < len(row[0]) < MAX_LENGTH) and (MIN_LENGTH < len(row[1]) < MAX_LENGTH)):
          dataset.append([row[0], row[1]])
  return dataset[:30000] #reducing dataset to save memory
                           



In [0]:
chosen_dataset = choose_conv_by_length()

In [0]:
#creating dictionaries

def create_dicts():

  #encoding dictionaries
  questions_dictionary_word_to_idx = {}
  answers_dictionary_word_to_idx = {}

  #decoding dictionaries
  questions_dictionary_idx_to_word = {}
  answers_dictionary_idx_to_word  = {}

  question_idx = 1
  answer_idx = 1

  for row in chosen_dataset:
      for word in row[0]:
        if word not in questions_dictionary_word_to_idx:
          questions_dictionary_word_to_idx[word] = question_idx
          questions_dictionary_idx_to_word[question_idx] = word
          question_idx +=1
      for word in row[1]:
        if word not in answers_dictionary_word_to_idx:
          answers_dictionary_word_to_idx[word] = answer_idx
          answers_dictionary_idx_to_word[answer_idx] = word
          answer_idx +=1
          
  return (questions_dictionary_word_to_idx, answers_dictionary_word_to_idx, questions_dictionary_idx_to_word, answers_dictionary_idx_to_word)

In [0]:

questions_dictionary_word_to_idx, answers_dictionary_word_to_idx, questions_dictionary_idx_to_word, answers_dictionary_idx_to_word = create_dicts()



In [0]:
#encoding questions and answers and zero padding

def encode_questions_and_answers(dataset_to_encode):
  dataset = dataset_to_encode.copy()
  for row in dataset:
    for idx, word in enumerate(row[0]):
      row[0][idx] = questions_dictionary_word_to_idx[word]
    for idx, word in enumerate(row[1]):   
      row[1][idx] = answers_dictionary_word_to_idx[word]                    
  return dataset

def make_zero_padding(dataset_to_pad):
    dataset = dataset_to_pad[:]
    
    for row in dataset:
      if (len(row[0]) < MAX_LENGTH):
        zeros1 = (MAX_LENGTH-len(row[0]))*[0]
        row[0]= row[0] + zeros1
      if (len(row[1]) < MAX_LENGTH):
        zeros2 = (MAX_LENGTH-len(row[1])) * [0]
        row[1] = row[1] + zeros2
    return dataset

In [0]:
enc = encode_questions_and_answers(chosen_dataset)
pad = make_zero_padding(enc)

In [0]:
#importing TensorFlow and model

#import tensorflow as tf
from tensorflow.contrib import seq2seq

In [17]:
#create list for bucket sizes

buckets = len(pad) * [(MAX_LENGTH,MAX_LENGTH)]

print(buckets[0])

(25, 25)


In [0]:
model_dir = "/content/gdrive/My Drive/Chatbot"
model_checkpoints = model_dir + "/chatbot.ckpt"

This function calls the constructor of the model, passing the following parameters:

The source vocabulary size (German, in our example)
The target vocabulary size (English, in our example)
The buckets (in our example is just one, since we padded all the sequences to a single size)
The long short-term memory (LSTM) internal units size
The number of stacked LSTM layers
The maximum norm of the gradient (for gradient clipping)
The mini-batch size (that is, how many observations for each training step)
The learning rate
The learning rate decay factor
The direction of the model
The type of data (in our example, we will use flat16, that is, float using 2 bytes)

In [0]:
data_set = pad

[GitHub repo for the model](https://raw.githubusercontent.com/marcotcr/tf-models/master/tutorials/rnn/translate/seq2seq_model.py)

In [0]:
# Copyright 2015 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

"""Sequence-to-sequence model with an attention mechanism."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import random

import numpy as np
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf


class Seq2SeqModel(object):
  """Sequence-to-sequence model with attention and for multiple buckets.
  This class implements a multi-layer recurrent neural network as encoder,
  and an attention-based decoder. This is the same as the model described in
  this paper: http://arxiv.org/abs/1412.7449 - please look there for details,
  or into the seq2seq library for complete model implementation.
  This class also allows to use GRU cells in addition to LSTM cells, and
  sampled softmax to handle large output vocabulary size. A single-layer
  version of this model, but with bi-directional encoder, was presented in
    http://arxiv.org/abs/1409.0473
  and sampled softmax is described in Section 3 of the following paper.
    http://arxiv.org/abs/1412.2007
  """

  def __init__(self,
               source_vocab_size,
               target_vocab_size,
               buckets,
               size,
               num_layers,
               max_gradient_norm,
               batch_size,
               learning_rate,
               learning_rate_decay_factor,
               use_lstm=False,
               num_samples=512,
               forward_only=False,
               dtype=tf.float32):
    """Create the model.
    Args:
      source_vocab_size: size of the source vocabulary.
      target_vocab_size: size of the target vocabulary.
      buckets: a list of pairs (I, O), where I specifies maximum input length
        that will be processed in that bucket, and O specifies maximum output
        length. Training instances that have inputs longer than I or outputs
        longer than O will be pushed to the next bucket and padded accordingly.
        We assume that the list is sorted, e.g., [(2, 4), (8, 16)].
      size: number of units in each layer of the model.
      num_layers: number of layers in the model.
      max_gradient_norm: gradients will be clipped to maximally this norm.
      batch_size: the size of the batches used during training;
        the model construction is independent of batch_size, so it can be
        changed after initialization if this is convenient, e.g., for decoding.
      learning_rate: learning rate to start with.
      learning_rate_decay_factor: decay learning rate by this much when needed.
      use_lstm: if true, we use LSTM cells instead of GRU cells.
      num_samples: number of samples for sampled softmax.
      forward_only: if set, we do not construct the backward pass in the model.
      dtype: the data type to use to store internal variables.
    """
    self.source_vocab_size = source_vocab_size
    self.target_vocab_size = target_vocab_size
    self.buckets = buckets
    self.batch_size = batch_size
    self.learning_rate = tf.Variable(
        float(learning_rate), trainable=False, dtype=dtype)
    self.learning_rate_decay_op = self.learning_rate.assign(
        self.learning_rate * learning_rate_decay_factor)
    self.global_step = tf.Variable(0, trainable=False)

    # If we use sampled softmax, we need an output projection.
    output_projection = None
    softmax_loss_function = None
    # Sampled softmax only makes sense if we sample less than vocabulary size.
    if num_samples > 0 and num_samples < self.target_vocab_size:
      w_t = tf.get_variable("proj_w", [self.target_vocab_size, size], dtype=dtype)
      w = tf.transpose(w_t)
      b = tf.get_variable("proj_b", [self.target_vocab_size], dtype=dtype)
      output_projection = (w, b)

      def sampled_loss(labels, inputs):
        labels = tf.reshape(labels, [-1, 1])
        # We need to compute the sampled_softmax_loss using 32bit floats to
        # avoid numerical instabilities.
        local_w_t = tf.cast(w_t, tf.float32)
        local_b = tf.cast(b, tf.float32)
        local_inputs = tf.cast(inputs, tf.float32)
        return tf.cast(
            tf.nn.sampled_softmax_loss(
                weights=local_w_t,
                biases=local_b,
                labels=labels,
                inputs=local_inputs,
                num_sampled=num_samples,
                num_classes=self.target_vocab_size),
            dtype)
      softmax_loss_function = sampled_loss

    # Create the internal multi-layer cell for our RNN.
    def single_cell():
      #gated recurrent unit
      return tf.contrib.rnn.GRUCell(size)
    if use_lstm:
      #long short term memory
      def single_cell():
        return tf.contrib.rnn.BasicLSTMCell(size)
    cell = single_cell()
    if num_layers > 1:
      cell = tf.contrib.rnn.MultiRNNCell([single_cell() for _ in range(num_layers)])

    # The seq2seq function: we use embedding for the input and attention.
    def seq2seq_f(encoder_inputs, decoder_inputs, do_decode):
      return tf.contrib.legacy_seq2seq.embedding_attention_seq2seq(
          encoder_inputs,
          decoder_inputs,
          cell,
          num_encoder_symbols=source_vocab_size,
          num_decoder_symbols=target_vocab_size,
          embedding_size=size,
          output_projection=output_projection,
          feed_previous=do_decode,
          dtype=dtype)

    # Feeds for inputs.
    self.encoder_inputs = []
    self.decoder_inputs = []
    self.target_weights = []
    for i in xrange(buckets[-1][0]):  # Last bucket is the biggest one.
      self.encoder_inputs.append(tf.placeholder(tf.int32, shape=[None],
                                                name="encoder{0}".format(i)))
    for i in xrange(buckets[-1][1] + 1):
      self.decoder_inputs.append(tf.placeholder(tf.int32, shape=[None],
                                                name="decoder{0}".format(i)))
      self.target_weights.append(tf.placeholder(dtype, shape=[None],
                                                name="weight{0}".format(i)))

    # Our targets are decoder inputs shifted by one.
    targets = [self.decoder_inputs[i + 1]
               for i in xrange(len(self.decoder_inputs) - 1)]

    # Training outputs and losses.
    if forward_only:
      self.outputs, self.losses = tf.contrib.legacy_seq2seq.model_with_buckets(
          self.encoder_inputs, self.decoder_inputs, targets,
          self.target_weights, buckets, lambda x, y: seq2seq_f(x, y, True),
          softmax_loss_function=softmax_loss_function)
      # If we use output projection, we need to project outputs for decoding.
      if output_projection is not None:
        for b in xrange(len(buckets)):
          self.outputs[b] = [
              tf.matmul(output, output_projection[0]) + output_projection[1]
              for output in self.outputs[b]
          ]
    else:
      self.outputs, self.losses = tf.contrib.legacy_seq2seq.model_with_buckets(
          self.encoder_inputs, self.decoder_inputs, targets,
          self.target_weights, buckets,
          lambda x, y: seq2seq_f(x, y, False),
          softmax_loss_function=softmax_loss_function)

    # Gradients and SGD update operation for training the model.
    params = tf.trainable_variables()
    if not forward_only:
      self.gradient_norms = []
      self.updates = []
      opt = tf.train.GradientDescentOptimizer(self.learning_rate)
      for b in xrange(len(buckets)):
        gradients = tf.gradients(self.losses[b], params)
        clipped_gradients, norm = tf.clip_by_global_norm(gradients,
                                                         max_gradient_norm)
        self.gradient_norms.append(norm)
        self.updates.append(opt.apply_gradients(
            zip(clipped_gradients, params), global_step=self.global_step))

    self.saver = tf.train.Saver(tf.global_variables())

  def step(self, session, encoder_inputs, decoder_inputs, target_weights,
           bucket_id, forward_only):
    """Run a step of the model feeding the given inputs.
    Args:
      session: tensorflow session to use.
      encoder_inputs: list of numpy int vectors to feed as encoder inputs.
      decoder_inputs: list of numpy int vectors to feed as decoder inputs.
      target_weights: list of numpy float vectors to feed as target weights.
      bucket_id: which bucket of the model to use.
      forward_only: whether to do the backward step or only forward.
    Returns:
      A triple consisting of gradient norm (or None if we did not do backward),
      average perplexity, and the outputs.
    Raises:
      ValueError: if length of encoder_inputs, decoder_inputs, or
        target_weights disagrees with bucket size for the specified bucket_id.
    """
    # Check if the sizes match.
    encoder_size, decoder_size = self.buckets[bucket_id]
    if len(encoder_inputs) != encoder_size:
      raise ValueError("Encoder length must be equal to the one in bucket,"
                       " %d != %d." % (len(encoder_inputs), encoder_size))
    if len(decoder_inputs) != decoder_size:
      raise ValueError("Decoder length must be equal to the one in bucket,"
                       " %d != %d." % (len(decoder_inputs), decoder_size))
    if len(target_weights) != decoder_size:
      raise ValueError("Weights length must be equal to the one in bucket,"
                       " %d != %d." % (len(target_weights), decoder_size))

    # Input feed: encoder inputs, decoder inputs, target_weights, as provided.
    input_feed = {}
    for l in xrange(encoder_size):
      input_feed[self.encoder_inputs[l].name] = encoder_inputs[l]
    for l in xrange(decoder_size):
      input_feed[self.decoder_inputs[l].name] = decoder_inputs[l]
      input_feed[self.target_weights[l].name] = target_weights[l]

    # Since our targets are decoder inputs shifted by one, we need one more.
    last_target = self.decoder_inputs[decoder_size].name
    input_feed[last_target] = np.zeros([self.batch_size], dtype=np.int32)

    # Output feed: depends on whether we do a backward step or not.
    if not forward_only:
      output_feed = [self.updates[bucket_id],  # Update Op that does SGD.
                     self.gradient_norms[bucket_id],  # Gradient norm.
                     self.losses[bucket_id]]  # Loss for this batch.
    else:
      output_feed = [self.losses[bucket_id]]  # Loss for this batch.
      for l in xrange(decoder_size):  # Output logits.
        output_feed.append(self.outputs[bucket_id][l])

    outputs = session.run(output_feed, input_feed)
    if not forward_only:
      return outputs[1], outputs[2], None  # Gradient norm, loss, no outputs.
    else:
      return None, outputs[0], outputs[1:]  # No gradient norm, loss, outputs.

  def get_batch(self, data, bucket_id):
    """Get a random batch of data from the specified bucket, prepare for step.
    To feed data in step(..) it must be a list of batch-major vectors, while
    data here contains single length-major cases. So the main logic of this
    function is to re-index data cases to be in the proper format for feeding.
    Args:
      data: a tuple of size len(self.buckets) in which each element contains
        lists of pairs of input and output data that we use to create a batch.
      bucket_id: integer, which bucket to get the batch for.
    Returns:
      The triple (encoder_inputs, decoder_inputs, target_weights) for
      the constructed batch that has the proper format to call step(...) later.
    """
    encoder_size, decoder_size = self.buckets[bucket_id]
    encoder_inputs, decoder_inputs = [], []

    # Get a random batch of encoder and decoder inputs from data,
    # pad them if needed, reverse encoder inputs and add GO to decoder.
    for _ in xrange(self.batch_size):
      encoder_input, decoder_input = random.choice(data[bucket_id])
      
      # Encoder inputs are reversed.
      encoder_inputs.append(list(reversed(encoder_input)))

      # Decoder inputs 
      decoder_inputs.append(decoder_input)

    # Now we create batch-major vectors from the data selected above.
    batch_encoder_inputs, batch_decoder_inputs, batch_weights = [], [], []

    # Batch encoder inputs are just re-indexed encoder_inputs.
    for length_idx in xrange(encoder_size):
      batch_encoder_inputs.append(
          np.array([encoder_inputs[batch_idx][length_idx]
                    for batch_idx in xrange(self.batch_size)], dtype=np.int32))

    # Batch decoder inputs are re-indexed decoder_inputs, we create weights.
    for length_idx in xrange(decoder_size):
      batch_decoder_inputs.append(
          np.array([decoder_inputs[batch_idx][length_idx]
                    for batch_idx in xrange(self.batch_size)], dtype=np.int32))

      # Create target_weights to be 0 for targets that are padding.
      batch_weight = np.ones(self.batch_size, dtype=np.float32)
      for batch_idx in xrange(self.batch_size):
        # We set weight to 0 if the corresponding target is a PAD symbol.
        # The corresponding target is decoder_input shifted by 1 forward.
        if length_idx < decoder_size - 1:
          target = decoder_inputs[batch_idx][length_idx + 1]
        if length_idx == decoder_size - 1 or target == 0:
          batch_weight[batch_idx] = 0.0
      batch_weights.append(batch_weight)
    return batch_encoder_inputs, batch_decoder_inputs, batch_weights

In [0]:
tf.reset_default_graph() 

In [0]:
#defining the sequence to sequence model

def get_seq2seq_model(session, forward_only, model_dir):
    model = Seq2SeqModel(
            source_vocab_size=len(questions_dictionary_word_to_idx),
            target_vocab_size=len(answers_dictionary_word_to_idx),
            buckets=buckets,
            size=64,
            num_layers=2,
            max_gradient_norm=5.0,
            batch_size=64,
            learning_rate=0.5,
            learning_rate_decay_factor=0.99,
            forward_only=forward_only)
    ckpt = tf.train.get_checkpoint_state(model_dir)
    if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
        print("Reading model parameters from {}".format(ckpt.model_checkpoint_path))
        model.saver.restore(session, ckpt.model_checkpoint_path)
    else:
        print("Created model with fresh parameters.")
        session.run(tf.global_variables_initializer())
    return model

In [0]:
def train():
    with tf.Session() as sess:
        model = get_seq2seq_model(sess, False, model_dir)
        # This is the training loop.
        step_time, loss = 0.0, 0.0
        current_step = 0
        bucket = 0
        steps_per_checkpoint = 100
        max_steps = 20000
        while current_step < max_steps:
            start_time = time.time()
            encoder_inputs, decoder_inputs, target_weights = model.get_batch([data_set], bucket)
            _, step_loss, _ = model.step(sess, encoder_inputs, decoder_inputs, target_weights, bucket, False)
            step_time += (time.time() - start_time) / steps_per_checkpoint
            loss += step_loss / steps_per_checkpoint
            current_step += 1
            if current_step % steps_per_checkpoint == 0:
                perplexity = math.exp(float(loss)) if loss < 300 else float("inf")
                print ("global step {} learning rate {} step-time {} perplexity {}".format(
                model.global_step.eval(), model.learning_rate.eval(), step_time, perplexity))
                sess.run(model.learning_rate_decay_op)
                model.saver.save(sess, model_checkpoints, global_step=model.global_step)
                step_time, loss = 0.0, 0.0
                encoder_inputs, decoder_inputs, target_weights = model.get_batch([data_set], bucket)
                _, eval_loss, _ = model.step(sess, encoder_inputs, decoder_inputs, target_weights, bucket, True)
                eval_ppx = math.exp(float(eval_loss)) if eval_loss < 300 else float("inf")
                print(" eval: perplexity {}".format(eval_ppx))
                sys.stdout.flush()  

In [0]:

train()